In [222]:
import math
import numpy as np
import sympy as sp
import scipy
import IPython.display as dp

In [3]:
x = sp.symbols('x')
f = x**2 * sp.sin((x * sp.pi) / 2)
a = -1
b = 1

In [215]:
# X = [(a + k * (b - a)/2) for k in range(3)]
# Y = [f.subs(x, x_k) for x_k in X]
X = [0, 1/2, 1]
Y = [0, 1, -1]
print(f"x_k = {X}")
print(f"y_k = {Y}")

x_k = [0, 0.5, 1]
y_k = [0, 1, -1]


In [228]:
def get_matrixes(X, Y):
    nsegs = len(X) - 1 # Number of segments
    n = 4*(nsegs)
    cub   = lambda x, y: np.array([[1, x, x**2, x**3],
                                   [1, y, y**2, y**3]])
    dcub  = lambda x: np.array([0, 1, 2*x, 3*x**2, 0, -1, -2*x, -3*x**2])
    ddcub = lambda x: np.array([0, 0, 2, 6*x])

    # Добавление условий интерполяции
    int_rules = np.zeros((nsegs, 2, n))
    sample = np.zeros((2, n-4))
    for k in range(nsegs):
        int_rules[k] = np.insert(sample, 4 * k, cub(X[k], X[k+1]).T, axis=1)
    int_rules = np.reshape(int_rules, (int_rules.size))

    # Условия гладкой склейки на внутренних узловых точках
    sm_rules = np.zeros((nsegs-1, n))  # Smooth gluing rules
    sample = np.zeros((n-8, ))
    for k in range(nsegs-1):
        sm_rules[k] = np.insert(sample, 4*k, dcub(X[k+1]).T)

    # Условия дважды гладкой склейки на внутренних узловых точках
    dsm_rules = np.zeros((nsegs-1, n))
    for k in range(nsegs-1):
        dsm_rules[k] = np.insert(sample, 4*k, np.append(ddcub(X[k+1]), ddcub(-X[k+1])).T)
    
    # Добавление остаточных условий
    res = np.array(ddcub(X[0]))
    res.resize(2 * n - 4)
    res = np.append(res, ddcub(X[-1]))


    # Объединение всех условий в одну матрицу коэф СЛАУ
    int_rules = np.append(int_rules, [sm_rules, dsm_rules])
    res = np.append(int_rules, res)
    res.resize((n, n))

    # Добавление вектора-столбца свободных членов СЛАУ
    resY = np.zeros(n)
    for k in range(nsegs):
        resY[2*k] = Y[k]
        resY[2*k+1] = Y[k+1]
        
    return [res, resY]

In [229]:
M = get_matrixes(X, Y)

Решение СЛАУ

In [250]:
solution = np.linalg.solve(M[0], M[1])
solution = np.reshape(solution, (len(X)-1, 4))
solution

array([[  0. ,   6.5,   0. , -18. ],
       [  7.5, -20.5,  18. ,  -6. ]])

In [251]:
a = np.array([1, x, x**2, x**3])
solution = np.dot(solution, a.T)[:, None]
solution

array([[-18.0*x**3 + 6.5*x],
       [-6.0*x**3 + 18.0*x**2 - 20.5*x + 7.5]], dtype=object)

In [254]:
for sol in solution:
    dp.display(dp.Markdown(rf"${sp.latex(sol[0])}$"))

$- 18.0 x^{3} + 6.5 x$

$- 6.0 x^{3} + 18.0 x^{2} - 20.5 x + 7.5$

In [256]:
sp.latex(solution[0][0])

'- 18.0 x^{3} + 6.5 x'